In [5]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
from torchvision.datasets.utils import download_url
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as functions
import tarfile
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import numpy as np
import pandas as pd

In [6]:
#Importing all of the Stock data found at "https://www.nasdaq.com/market-activity/quotes/historical". I am just using opening data
df1 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760112203.csv', sep = ',', header = None)
openingAAPL = df1[3].str.replace('$', '')
openingAAPL = openingAAPL.drop(0)
openingAAPL = openingAAPL.apply(float)

In [7]:
df2 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760148202.csv', sep = ',', header = None)
openingSUBX = df2[3].str.replace('$', '')
openingSUBX = openingSUBX.drop(0)
openingSUBX = openingSUBX.apply(float)

In [8]:
df3 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760169825.csv', sep = ',', header = None)
openingMSFT = df3[3].str.replace('$', '')
openingMSFT = openingMSFT.drop(0)
openingMSFT = openingMSFT.apply(float)

In [9]:
df4 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760179851.csv', sep = ',', header = None)
openingCSCO = df4[3].str.replace('$', '')
openingCSCO = openingCSCO.drop(0)
openingCSCO = openingCSCO.apply(float)

In [10]:
df5 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760200748.csv', sep = ',', header = None)
openingQCOM = df5[3].str.replace('$', '')
openingQCOM = openingQCOM.drop(0)
openingQCOM = openingQCOM.apply(float)

In [11]:
df6 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760225227.csv', sep = ',', header = None)
openingMETA = df6[3].str.replace('$', '')
openingMETA = openingMETA.drop(0)
openingMETA = openingMETA.apply(float)

In [12]:
df7 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760253051.csv', sep = ',', header = None)
openingAMZN = df7[3].str.replace('$', '')
openingAMZN = openingAMZN.drop(0)
openingAMZN = openingAMZN.apply(float)

In [13]:
df8 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760263750.csv', sep = ',', header = None)
openingTSLA = df8[3].str.replace('$', '')
openingTSLA = openingTSLA.drop(0)
openingTSLA = openingTSLA.apply(float)

In [14]:
df9 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760263750.csv', sep = ',', header = None)
openingAMD = df9[3].str.replace('$', '')
openingAMD = openingAMD.drop(0)
openingAMD = openingAMD.apply(float)

In [15]:
df10 = pd.read_csv('/Users/valendunn/Desktop/FPGA-ML/ML Base/HistoricalData_1727760293784.csv', sep = ',', header = None)
openingNFLX = df10[3].str.replace('$', '')
openingNFLX = openingNFLX.drop(0)
openingNFLX = openingNFLX.apply(float)


In [16]:
#Put them all in one dataset to go through all of the info
openingALL = pd.concat([openingAAPL, openingSUBX, openingMSFT, openingCSCO, openingQCOM, openingMETA, openingAMZN, openingTSLA, openingAMD, openingNFLX], axis =1 )
print(openingALL)

             3      3        3      3       3       3         3         3  \
1     230.0400  97.39  428.210  53.02  171.37  567.70  187.1400  259.0400   
2     228.4600  97.65  431.520  53.04  173.00  570.10  190.6800  257.3750   
3     227.3000  98.50  435.085  53.00  174.91  575.73  194.3100  260.6000   
4     224.9300  94.26  429.830  52.57  166.28  564.05  193.7500  252.5400   
5     228.6450  95.47  433.000  52.21  167.42  566.68  194.2700  254.0800   
...        ...    ...      ...    ...     ...     ...       ...       ...   
1255   56.4100  84.90  136.750  47.25   76.32  179.55   86.3010   15.4407   
1256   54.6075  84.30  134.950  46.32   73.93  175.57   85.6500   15.4573   
1257   55.7650  85.95  136.250  47.35   75.26  174.84   86.3870   16.2193   
1258   56.2675  88.63  139.660  49.65   77.06  179.15   87.3000   16.1000   
1259   55.2250  88.41  138.050  48.95   76.89  177.87   86.3495   16.2000   

             3        3  
1     259.0400  707.000  
2     257.3750  708.360

In [44]:
#turning the pd data into a tensor dataset
l = openingAAPL.size
i = 1
dataset = torch.zeros(11920, 150)
labels = torch.zeros(11920,1)
num = 0
k = 0
while(k < 10):
    i = 0
    while(i+67 < l):
        labels[num] = openingALL.iloc[i,k]
        entry = torch.zeros(1,150)
        j = 0
        while(j < 60):
            entry[0,j] = openingALL.iloc[(i+7+j), k]
            j+= 1
        c = 0
        entryn = 60
        while(c < 10):
            if(c != k):
                t = 0
                while(t < 10):
                    entry[0,entryn] = openingALL.iloc[(i+7+t),c]
                    entryn += 1
                    t += 1
            c+= 1
        dataset[num] = entry
        num += 1
        i+= 1
    k+= 1

In [36]:
#testing how to set up each entry
entry = torch.zeros(1,150)
j = 0
while(j <= 60):
    entry[0,j] = openingALL.iloc[(i+7+j), k]
    j+= 1
c = 0
entryn = 60
while(c < 10):
    if(c != k):
        t = 0
        while(t < 10):
            entry[0,entryn] = openingALL.iloc[(i+7+t),c]
            entryn += 1
            t += 1
    c+= 1

In [45]:
#printing to check
print(dataset)
print(labels)

1192*10

tensor([[224.9900, 217.5500, 215.7500,  ..., 676.8700, 674.0000, 681.8600],
        [217.5500, 215.7500, 216.5400,  ..., 674.0000, 681.8600, 673.1400],
        [215.7500, 216.5400, 223.5800,  ..., 681.8600, 673.1400, 673.3100],
        ...,
        [332.9600, 334.6000, 334.0100,  ...,  25.2660,  24.1700,  24.0700],
        [334.6000, 334.0100, 337.7600,  ...,  24.1700,  24.0700,  23.6613],
        [334.0100, 337.7600, 335.0000,  ...,  24.0700,  23.6613,  23.4584]])
tensor([[230.0400],
        [228.4600],
        [227.3000],
        ...,
        [331.4900],
        [336.4700],
        [323.1200]])


11920

In [51]:
findata =torch.utils.data.TensorDataset(dataset, labels)

train_len = 0.8 * len(findata)
test_len = len(findata) - train_len

print(test_len + train_len)
print(len(findata))

train_set, test_set = torch.utils.data.random_split(findata, [int(train_len), int(test_len)])

11920.0
11920


In [114]:
#Setting up the data for seperate or unified training
aapldata = findata[0:1191]
subxdata = findata[1192:2383]
msftdata = findata[2384:3575]
cscodata = findata[3576:4767]
qcomdata = findata[4768:5959]
metadata = findata[5960:7151]
amzndata = findata[7152:8343]
tsladata = findata[8344:9535]
amddata = findata[9536:10727]
nflxdata = findata[10728:11919]
